In [1]:
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import os
from scipy.misc import imread
from scipy.misc import imresize
from Proj_iTraker import UI
import cv2
sess=tf.InteractiveSession()
cumt_data=np.load('cumt_data.pkl')


In [2]:
X=tf.placeholder(shape=[None,32,128,3],dtype=tf.float32)
Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
DROPOUT=tf.placeholder(dtype=tf.float32)
LR=tf.placeholder(dtype=tf.float32)
BN_FLAG=tf.placeholder(dtype=tf.bool)

In [3]:
with tf.name_scope('model'):
    model=Cumt_itraker(image=X,bn_flag=BN_FLAG,dropout_rate=DROPOUT)
    y_score=model.score

Cumt_itraker Init Ready!


In [4]:
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)

In [5]:
writer_tr=tf.summary.FileWriter('./mylog/test')
writer_te=tf.summary.FileWriter('./mylog/val')
merge=tf.summary.merge_all()
ex_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)



In [ ]:
#初始化变量，需要重新训练才运行
init=tf.global_variables_initializer()
sess.run(init)

In [6]:
#从保存的检查点钟复原变量
saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_71.ckpt-64600')

INFO:tensorflow:Restoring parameters from ./model_save/model_71.ckpt-64600


In [ ]:
test_range=np.random.choice(range(cumt_data['data'].shape[0]),int(cumt_data['data'].shape[0]*0.2),replace=False)
train_range=np.asarray(list(set(range(cumt_data['data'].shape[0]))-set(test_range)))

In [ ]:
d=np.load('eye_val.pkl')
val={}
val['images']=d['images'].astype('float32')-[103.939, 116.779, 123.68]
val['labels']=d['labels']
del d

In [ ]:
saver=tf.train.Saver()

In [ ]:
lr_=1e-4
for i in range(1000*6*12):
    mask=np.random.choice(train_range,256,replace=False)
    x_,y_=cumt_data['data'][mask],cumt_data['labels'][mask]
    feed_dict={X:x_,Y:y_,DROPOUT:0.3,BN_FLAG:True,LR:lr_}
    sess.run([TRAIN,ex_ops],feed_dict=feed_dict)
#     if i%10==0:
#         loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
#         writer_tr.add_summary(m_,i)
#         print('epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
    if i%50==0:
        mask=np.random.choice(test_range,256,replace=False)
        x_,y_=cumt_data['data'][mask],cumt_data['labels'][mask]
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
    if i%10==0:
        x_,y_=val['images'],val['labels']
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        if acc_>0.7:
            m_name='./model_save/model2_'+str(int(acc_*100))+'.ckpt'
            saver.save(sess,m_name,global_step=i)
            lr_=max(1e-5,0.97*lr_)
            print('Save model,epoch:{},validate accuracy:{},learning rate:{}'.format(i,acc_,lr_))
        print('--epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model.ckpt',global_step=1000)

In [ ]:
'''
在视频中插入判别网络
'''

In [10]:
prob=tf.nn.softmax(y_score,name='prob')
y_guess=tf.arg_max(prob,1)+1

In [11]:
labmap={10:1,
        1:0,2:9,3:8,
        4:7,5:6,6:5,
        7:4,8:3,9:2, 
}

In [27]:
cap = cv2.VideoCapture('test.flv')
while (True):
    ret,frame = cap.read()
    if ret==True:
        #ori 为原始图片--bgr,网络训练是用rgb训练的
        ori_img=frame.copy()
        #截取眼部图片
        eye_img=UI.drew_face_eye(frame,minNeighbors=7,minSize=(20,20),reduce_pix=10)
        
        if eye_img is None:
            #没有检测到眼睛，跳到下一帧
            cv2.imshow('frame',ori_img)
            continue
        eye_img=(eye_img-[103.939, 116.779, 123.68])[:,:,::-1]
        #检测到眼睛，送入网络
        x_=imresize(eye_img,(32,128,3)).reshape((1,32,128,3))
        pre_asw=sess.run(y_guess,feed_dict={X:x_,DROPOUT:1.,BN_FLAG:False})[0]
        #给图片划线，划框
        frame=UI.drawline(frame,line_w=1)
        print(labmap[pre_asw])
        if pre_asw == 1:
            #闭眼的情况
#             UI.drawblock(frame,1)
#             UI.drawblock(frame,2)
#             UI.drawblock(frame,3)
            pass
        else:
            #有检测结果
            frame=UI.drawblock(frame,labmap[pre_asw])
            #cv2.waitKey()
        #cv2.imshow('frame',x_.reshape((32,128,3)))
        cv2.imshow('frame',frame)
        if cv2.waitKey(300)& 0xff ==ord('q'):
            print('quite')
            break
    else:
        print('video end')
        break
cap.release()
cv2.destroyAllWindows()

0
0
0
0
0
0
8
5
5
5
5
5
5
5
5
5
0
0
2
0
0
0
0
0
0
0
0
0
5
5
5
0
0
0
0
0
0
9
0
0
0
0
0
5
0
5
0
0
0
7
0
0
0
2
0
5
2
5
5
5
5
5
5
5
5
5
5
5
0
0
2
2
0
0
0
0
0
0
0
0
5
5
2
0
0
0
0
0
0
0
0
0
0
0
0
9
5
8
8
8
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
8
5
5
5
5
2
5
8
5
5
0
0
8
5
5
5
5
8
5
5
5
5
5
5
0
0
0
8
8
8
0
8
0
0
0
0
0
0
0
2
0
0
2
2
2
0
2
5
5
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
2
2
5
5
5
5
0
0
0
0
0
0
0
0
5
0
0
0
0
5
quite
